In [1]:
import tensorflow as tf

In [4]:
!pip install -U tensorflow-addons

     |████████████████████████████████| 679 kB 14.4 MB/s 


In [5]:
import tensorflow_addons as tfa

In [59]:
mnist= tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test)= mnist.load_data()

x_train, x_test= x_train/255.0, x_test/255.0

In [34]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [60]:
y_train

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [44]:
import numpy as np
import os

In [61]:
demx=[]
demy=[]

for i in range (len(y_train)):
  if (y_train[i]==1) or (y_train[i]==2):
    demx.append(x_train[i])
    demy.append(y_train[i])
    
demx=np.asarray(demx)
demy=np.asarray(demy)

In [62]:
demx.shape

(12700, 28, 28)

In [63]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
demx = demx.reshape(demx.shape[0], 28, 28, 1)

In [64]:
x_train = tf.image.resize(x_train, [14,14])
x_test = tf.image.resize(x_test, [14,14])

In [65]:
x_train.shape

TensorShape([60000, 14, 14, 1])

In [66]:
demx.shape

(12700, 28, 28, 1)

In [69]:
def create_model():
  model = tf.keras.models.Sequential([
  tfa.layers.SpatialPyramidPooling2D([1,4,12]),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
  ])

  model.compile(
    optimizer=tf.keras.optimizers.Adam(beta_1=0.9, beta_2=0.999),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
  ) 

  return model

In [70]:
model=create_model()
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
model.fit(
    x_train,y_train, epochs=10, validation_data=(x_test,y_test),callbacks=[cp_callback])
model.summary()

!mkdir -p saved_model
model.save('saved_model/my_model')


#model.evaluate(x_test, y_test, verbose=2)

Epoch 1/10


1875/1875 [==============================] - 5s 2ms/step - loss: 0.4165 - sparse_categorical_accuracy: 0.8838 - val_loss: 0.2008 - val_sparse_categorical_accuracy: 0.9433

Epoch 00001: saving model to training_1/cp.ckpt
Epoch 2/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2045 - sparse_categorical_accuracy: 0.9394 - val_loss: 0.1442 - val_sparse_categorical_accuracy: 0.9579

Epoch 00002: saving model to training_1/cp.ckpt
Epoch 3/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1569 - sparse_categorical_accuracy: 0.9541 - val_loss: 0.1172 - val_sparse_categorical_accuracy: 0.9667

Epoch 00003: saving model to training_1/cp.ckpt
Epoch 4/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1328 - sparse_categorical_accuracy: 0.9609 - val_loss: 0.1000 - val_sparse_categorical_accuracy: 0.9716

Epoch 00004: saving model to training_1/cp.ckpt
Epoch 5/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1164 - sp

INFO:tensorflow:Assets written to: saved_model/my_model/assets


INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [52]:
ls saved_model


my_model/


In [54]:
ls saved_model/my_model

assets/  keras_metadata.pb  saved_model.pb  variables/


In [55]:
os.listdir(checkpoint_dir)

['cp.ckpt.index', 'checkpoint', 'cp.ckpt.data-00000-of-00001']

In [71]:
#new model instance
model=create_model()
loss, acc = model.evaluate(demx, demy, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))
model.load_weights(checkpoint_path)
# Re-evaluate the model
loss, acc = model.evaluate(demx, demy, verbose=2)
print("dem x, dem y shape: ", demx.shape, demy.shape)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

397/397 - 1s - loss: 2.0768 - sparse_categorical_accuracy: 0.0954
Untrained model, accuracy:  9.54%
397/397 - 1s - loss: 0.0454 - sparse_categorical_accuracy: 0.9873
dem x, dem y shape:  (12700, 28, 28, 1) (12700,)
Restored model, accuracy: 98.73%


In [56]:
new_model = tf.keras.models.load_model('saved_model/my_model')

new_model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
spatial_pyramid_pooling2d_10 (None, 161, 1)            0         
_________________________________________________________________
flatten_12 (Flatten)         (None, 161)               0         
_________________________________________________________________
dense_24 (Dense)             (None, 128)               20736     
_________________________________________________________________
dropout_12 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_25 (Dense)             (None, 10)                1290      
Total params: 22,026
Trainable params: 22,026
Non-trainable params: 0
_________________________________________________________________
